<img src="images/img.png" />

# CS5228 Project, Group 32

## Data Preprocessing
In this part, we are going to perform some data preprocessing steps. This may include:
* Data cleaning: handle missing values, duplicates, inconsistant or invalid vallues, outliers

* Data reduction: reduce number of attributes, reduce number of attribute values

* Data transformation: attribute construction, normalization

* Data discretization: encode to numerical attributes

### Setting up the Notebook

In [1]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load file into pandas dataframe
df = pd.read_csv('./data/train.csv')

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points in training data, each with 30 attributes.


### Data Cleaning

Before data cleaning, remove the known attributes that are not meaningful to our prediction model:
  * Meaningless idendifier: listing_id 
  * Attributes in free text: title, description, features, accessories
  * Attribute with the same value: eco_category, indicative_price
  * Attribute unlikely to affect price: curb_weight

In [3]:
columns_to_drop = [
    'listing_id',          # Meaningless identifier
    'title',               # Attributes in free text
    'description',
    'features',
    'accessories',
    'eco_category',        # Attribute with the same value
    'indicative_price',
    'curb_weight',         # Attribute unlikely to affect price
    'original_reg_date',
    'lifespan',
]

df = df.drop(columns=columns_to_drop)

num_records, num_attributes = df.shape
print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points, each with 20 attributes.


### Handle Missing Values
Firstly, for each of the columns with missing value, check the number of rows with NaN values.
There are 3 scenarios:
1. NaN value is the major (e.g. fuel_type has 19121 rows with NaN values), we remove the corresponding attritubes.
2. NaN value is the minor. We can choose to fill or delete related data points. 

In [4]:
columns_to_check = [
    'make',
    'fuel_type',
    'manufactured',
    'power',
    'engine_cap',
    'mileage',
    'no_of_owners',
    'depreciation',
    'road_tax',
    'dereg_value',
    'omv',
    'arf',
    'opc_scheme'
]

# Calculate the number of NaN values in each specified column
nan_counts = df[columns_to_check].isna().sum()

# Print the number of NaN values for each column
print('-----Training data-----')
for column, count in nan_counts.items():
    print(f"Column '{column}' has {count} rows with NaN values.")

-----Training data-----
Column 'make' has 1316 rows with NaN values.
Column 'fuel_type' has 19121 rows with NaN values.
Column 'manufactured' has 7 rows with NaN values.
Column 'power' has 2640 rows with NaN values.
Column 'engine_cap' has 596 rows with NaN values.
Column 'mileage' has 5304 rows with NaN values.
Column 'no_of_owners' has 18 rows with NaN values.
Column 'depreciation' has 507 rows with NaN values.
Column 'road_tax' has 2632 rows with NaN values.
Column 'dereg_value' has 220 rows with NaN values.
Column 'omv' has 64 rows with NaN values.
Column 'arf' has 174 rows with NaN values.
Column 'opc_scheme' has 24838 rows with NaN values.


We delete attributes with TOO many NaN value here.

In [5]:
columns_to_drop_nan = [
    'fuel_type',
    'opc_scheme'
]

df = df.drop(columns=columns_to_drop_nan)

Then we try to fill up other missing values.

In [6]:
from util.DataPreprocess import HandlingMissingValues

df = HandlingMissingValues(df)

NaN values after handling:  0


### Remove Exact Duplicates
We remove duplicated data points here.

In [7]:
df = df.drop_duplicates()

num_records, num_attributes = df.shape
print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 24258 data points, each with 18 attributes.


### Merge rows with fewer data points on specific attributes

In [8]:
threshold = 2

value_counts = df['make'].value_counts()
categories_to_replace = value_counts[value_counts < threshold].index
df['make'] = df['make'].replace(categories_to_replace, 'others')

### Transform categorical value to numerical values

In [9]:
categorical_columns = [
    'make',
    'model',
    'type_of_vehicle',
    'transmission',
]

le = LabelEncoder()
for column in categorical_columns:
    df[column] = le.fit_transform(df[column])

### Transform date time attributes to numerical values

In [10]:
df['reg_date'] = pd.to_datetime(df['reg_date'], format='%d-%b-%Y')
df['reg_year'] = df['reg_date'].dt.year
df = df.drop(columns=['reg_date'])

num_records, num_attributes = df.shape
print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 24258 data points, each with 18 attributes.


### Handle category attribute

In [11]:
# Replace '-' with an empty string
df['category'] = df['category'].replace('-', '')

# Split the 'category' column into lists
df['category_list'] = df['category'].str.split(', ')

# Handle empty strings by replacing them with empty lists
df['category_list'] = df['category_list'].apply(lambda x: [] if x == [''] else x)

# Import itertools for flattening lists
from itertools import chain

# Flatten the list of lists to a single list
all_categories = list(chain.from_iterable(df['category_list']))

# Get the unique categories
unique_categories = set(all_categories)

# Print the number of unique categories
print(f"Number of unique categories: {len(unique_categories)}")
print("Unique categories:", unique_categories)

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the category lists
category_dummies = mlb.fit_transform(df['category_list'])

# Create a DataFrame with the one-hot encoded categories
category_df = pd.DataFrame(category_dummies, columns=mlb.classes_, index=df.index)

# Concatenate the new dummy columns to the original DataFrame
df = pd.concat([df, category_df], axis=1)

# Drop the temporary 'category_list' column if desired
df.drop('category_list', axis=1, inplace=True)
df.drop('category', axis=1, inplace=True)

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

Number of unique categories: 15
Unique categories: {'low mileage car', 'sgcarmart warranty cars', 'direct owner sale', 'opc car', 'imported used vehicle', 'sta evaluated car', 'premium ad car', 'almost new car', 'vintage cars', 'hybrid cars', 'parf car', 'electric cars', 'consignment car', 'coe car', 'rare & exotic'}
There are 24258 data points, each with 32 attributes.


### Saving the Data

In [12]:
file_name = './data/train_preprocessed.csv'

# Check if the file exists
if os.path.exists(file_name):
    # Delete the file
    os.remove(file_name)
    print(f"Existing file '{file_name}' has been deleted.")

# Save the DataFrame to CSV
df.to_csv(file_name, index=False)
print(f"DataFrame has been saved to '{file_name}'.")

Existing file './data/train_preprocessed.csv' has been deleted.
DataFrame has been saved to './data/train_preprocessed.csv'.


## Data Mining

### Load preprocessed training data

In [13]:
# Load file into pandas dataframe, we saved our preprocessed file at path 'output_file'
training_file = './data/train_preprocessed.csv'
df = pd.read_csv(training_file)

columns_to_keep = [
    'make',
    'model',
    'manufactured',
    'reg_year',
    'type_of_vehicle',
    'dereg_value',
    'depreciation',
    'power',
    'coe',
    'arf',
    'omv',
    'price',
    'road_tax'
]

df = df[columns_to_keep]
columns_to_keep = [col for col in df.columns if col != 'price']

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))

There are 24258 data points in training data, each with 13 attributes.


### Load test data

In [14]:
test_file = './data/test.csv'
df_test = pd.read_csv(test_file)

df_test['reg_date'] = pd.to_datetime(df_test['reg_date'], format='%d-%b-%Y')
df_test['reg_year'] = df_test['reg_date'].dt.year
df_test = df_test.drop(columns=['reg_date'])

num_records, num_attributes = df_test.shape
print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

categorical_columns = [
    'make',
    'model',
    'type_of_vehicle',
    'transmission',
]

for column in categorical_columns:
    df_test[column] = le.fit_transform(df_test[column])

df_test = df_test[columns_to_keep]


num_records, num_attributes = df_test.shape
print("There are {} data points in test data, each with {} attributes.". format(num_records, num_attributes))

There are 10000 data points, each with 29 attributes.
There are 10000 data points in test data, each with 12 attributes.


### Mining code here

In [15]:
from util.DataMining import split_dataframe, split_dataframe_flex
from util.DataMining import (
    RandomForestMining,
    RandomForestMiningByModel,
    GradientBoostingMining,
    LinearRegressionMining,
    LinearRegressionMiningByModel,
    CombinedDataMiningRandomForestAndLinearRegression
)

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [16]:
run_times, rmse_sum = 5, 0
for i in tqdm(range(run_times), desc='Running Random Forest'):
    target_col = 'price'
    x_train, x_test, y_train, y_test = split_dataframe(df, target_col)
    rmse_sum += RandomForestMining(x_train, x_test, y_train, y_test)
print('Average RMSE:', round(rmse_sum / run_times))

Running Random Forest:  20%|█████████████████▌                                                                      | 1/5 [00:26<01:44, 26.25s/it]

Running not in develop mode
RMSE on test data: 25451.98517130758


Running Random Forest:  40%|███████████████████████████████████▏                                                    | 2/5 [00:52<01:18, 26.19s/it]

Running not in develop mode
RMSE on test data: 27782.010908891385


Running Random Forest:  60%|████████████████████████████████████████████████████▊                                   | 3/5 [01:19<00:53, 26.64s/it]

Running not in develop mode
RMSE on test data: 19919.584039940735


Running Random Forest:  80%|██████████████████████████████████████████████████████████████████████▍                 | 4/5 [01:45<00:26, 26.41s/it]

Running not in develop mode
RMSE on test data: 18179.631341011085


Running Random Forest: 100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:11<00:00, 26.34s/it]

Running not in develop mode
RMSE on test data: 17300.451200995492
Average RMSE: 21727


In [17]:
x_train, y_train = df.drop(columns=['price']), df['price']
x_test = df_test[x_train.columns]

res = RandomForestMining(x_train, x_test, y_train, dev=True)
res.to_csv('./data/res.csv', index=False)